In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import pickle
import warnings
warnings.filterwarnings("ignore")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cve-models/authentication_randomforest_model.pkl
/kaggle/input/cve-models/accessComplexity_svm_model.pkl
/kaggle/input/cve-models/exploitabilityScore_ridge_regression_model.pkl
/kaggle/input/cve-models/impactScore_svm_model.pkl
/kaggle/input/cve-models/integratyImpact_xgboost_model.pkl
/kaggle/input/cve-models/baseScore_svr_model.pkl
/kaggle/input/cve-models/confidentialityImpact_label_encoder.pkl
/kaggle/input/cve-models/accessComplexity_label_encoder.pkl
/kaggle/input/cve-models/accessVector_support vector machine_model.pkl
/kaggle/input/cve-models/availabilityImpact_xgboost_model.pkl
/kaggle/input/cve-models/availabilityImpact_label_encoder.pkl
/kaggle/input/cve-models/confidentialityImpact_xgboost_model.pkl
/kaggle/input/cve-models/integrityImpact_label_encoder.pkl
/kaggle/input/cve-models/accessVector_label_encoder.pkl
/kaggle/input/cve-models/authentication_label_encoder.pkl


In [17]:
with open('/kaggle/input/cve-models/accessComplexity_label_encoder.pkl', 'rb') as file:
    accessComplexity_label_encoder = pickle.load(file)

with open('/kaggle/input/cve-models/accessComplexity_svm_model.pkl', 'rb') as file:
    accessComplexity_svm_model = pickle.load(file)

with open('/kaggle/input/cve-models/accessVector_label_encoder.pkl', 'rb') as file:
    accessVector_label_encoder = pickle.load(file)
    
with open('/kaggle/input/cve-models/accessVector_support vector machine_model.pkl', 'rb') as file:
    accessVector_svm_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/authentication_label_encoder.pkl', 'rb') as file:
    authentication_label_encoder = pickle.load(file)
    
with open('/kaggle/input/cve-models/authentication_randomforest_model.pkl', 'rb') as file:
    authentication_randomforest_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/availabilityImpact_label_encoder.pkl', 'rb') as file:
    availabilityImpact_label_encoder = pickle.load(file)
    
with open('/kaggle/input/cve-models/availabilityImpact_xgboost_model.pkl', 'rb') as file:
    availabilityImpact_xgboost_model = pickle.load(file)

with open('/kaggle/input/cve-models/confidentialityImpact_label_encoder.pkl', 'rb') as file:
    confidentialityImpact_label_encoder = pickle.load(file)
    
with open('/kaggle/input/cve-models/confidentialityImpact_xgboost_model.pkl', 'rb') as file:
    confidentialityImpact_xgboost_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/integrityImpact_label_encoder.pkl', 'rb') as file:
    integrityImpact_label_encoder = pickle.load(file)
    
with open('/kaggle/input/cve-models/integratyImpact_xgboost_model.pkl', 'rb') as file:
    integratyImpact_xgboost_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/baseScore_svr_model.pkl', 'rb') as file:
    baseScore_svr_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/impactScore_svm_model.pkl', 'rb') as file:
    impactScore_svm_model = pickle.load(file)
    
with open('/kaggle/input/cve-models/exploitabilityScore_ridge_regression_model.pkl', 'rb') as file:
    exploitabilityScore_ridge_regression_model = pickle.load(file)


In [10]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)

    # Get the hidden states from BERT
    with torch.no_grad():
        outputs = model(**inputs)

    embeddings = outputs.last_hidden_state[:, 0, :]  # Shape: (batch_size, hidden_size=768)
    
    return embeddings.squeeze().numpy()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [35]:
def get_valid_value(val):
    if val<0:
        return 0
    elif val>10:
        return 10
    else:
        return val

def find_all_outputs(text):
    
    embedding = np.array([get_bert_embeddings(text)])
    
    EXPLOITABILITY_SCORE = exploitabilityScore_ridge_regression_model.predict(embedding)
    print(f'Exploitability score is {get_valid_value(EXPLOITABILITY_SCORE)}')
    
    IMPACT_SCORE = impactScore_svm_model.predict(embedding)
    print(f'Impact score is {get_valid_value(IMPACT_SCORE)}')
    
    ACCESS_VECTOR = accessVector_svm_model.predict(embedding)
    ACCESS_VECTOR_CLASS = accessVector_label_encoder.inverse_transform(ACCESS_VECTOR)
    print(f'Access vector is {ACCESS_VECTOR_CLASS}')
    
    ACCESS_COMPLEXITY = accessComplexity_svm_model.predict(embedding)
    ACCESS_COMPLEXITY_CLASS = accessComplexity_label_encoder.inverse_transform(ACCESS_COMPLEXITY)
    print(f'Access complexity is {ACCESS_COMPLEXITY_CLASS}')
    
    CONFIDENTIALITY_IMPACT = confidentialityImpact_xgboost_model.predict(embedding)
    CONFIDENTIALITY_IMPACT_CLASS = confidentialityImpact_label_encoder.inverse_transform(CONFIDENTIALITY_IMPACT)
    print(f'Confidentiality impact is {CONFIDENTIALITY_IMPACT_CLASS}')
    
    INTEGRITY_IMPACT = integratyImpact_xgboost_model.predict(embedding)
    INTEGRITY_IMPACT_CLASS = integrityImpact_label_encoder.inverse_transform(INTEGRITY_IMPACT)
    print(f'Integrity impact is {INTEGRITY_IMPACT_CLASS}')
    
    AVAILABILITY_IMPACT = availabilityImpact_xgboost_model.predict(embedding)
    AVAILABILITY_IMPACT_CLASS = availabilityImpact_label_encoder.inverse_transform(AVAILABILITY_IMPACT)
    print(f'Availability impact is {AVAILABILITY_IMPACT_CLASS}')
    
    BASE_SCORE = baseScore_svr_model.predict(embedding)
    print(f'Impact score is {get_valid_value(BASE_SCORE)}')

In [8]:
input_text = 'The debug command in Sendmail is enabled, allowing attackers to execute commands as root.'

In [36]:
find_all_outputs(input_text)

Exploitability score is [9.59999852]
Impact score is [2]
Access vector is ['NETWORK']
Access complexity is ['LOW']
Confidentiality impact is ['COMPLETE']
Integrity impact is ['COMPLETE']
Availability impact is ['PARTIAL']
Impact score is [7.66870956]
